 # DrugBank

 Author: Moshe Silverstein <br/>
 Date: 03-2018 <br/>
 Data Source Home: http://www.drugbank.ca/ <br/>
 Data Source Download: https://www.drugbank.ca/releases/latest

 Reviewer: Charles Dai <br>
 Updated: 6-20

In [ ]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
import sys
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import harmonizome.utility_functions as uf
import harmonizome.lookup as lookup

In [ ]:
# from clustergrammer_widget import *
# net = Network(clustergrammer_widget)

In [ ]:
%load_ext autoreload
%autoreload 2

 ### Python Version

In [ ]:
sys.version

 # Initialization

 ### Options

In [ ]:
%%appyter code_eval

{% set group = ChoiceField(
    name='identifier',
    label='Protein Identifier Group',
    choices=['Target', 'Enzyme', 'Carrier', 'Transporter'],
    default='Target',
    section='data'
) %}

 ### Load Mapping Dictionaries

In [ ]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

 ### Output Path

In [ ]:
%%appyter code_exec

output_name = 'drugbank_' + '{{group}}'.lower()

path = 'Output/DrugBank-' + '{{group}}'
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
%%appyter hide_code
{% do SectionField(
    name='data',
    title='Load Data',
    subtitle='Upload Files from the DrugBank Protein Identifiers Data Sets',
) %}

 # Load Data

In [ ]:
%%appyter code_exec

df = pd.read_csv({{FileField(
    constraint='.*\.csv$',
    name='drug_identifiers', 
    label='Drug Identifiers Data Set', 
    default='Input/DrugBank/drugbank_all_target_polypeptide_ids.csv',
    section='data')
}}, usecols=['Gene Name', 'Drug IDs', 'Species'], index_col=0)

In [ ]:
df.head()

In [ ]:
df.shape

 # Load Drug Metadata

In [ ]:
%%appyter code_exec

drug_meta = pd.read_csv({{FileField(
    constraint='.*\.csv$',
    name='drug_metadata', 
    label='External Drug Links', 
    default='Input/DrugBank/drugbank_all_drug_links.csv',
    section='data')
}}, usecols=['DrugBank ID', 'Name'], index_col=0)

In [ ]:
drug_meta.head()

In [ ]:
drug_meta.shape

 # Pre-process Data

 ## Get Relevant Data

In [ ]:
# Get Relevant Species
df = df[np.logical_or.reduce([
    df['Species'] == 'Humans',
    df['Species'] == 'Mouse',
    df['Species'] == 'Rat'
])].drop('Species', axis=1)
df.head()

 ## Split Drug ID list

In [ ]:
df['Drug IDs'] = df['Drug IDs'].map(lambda x: x.split('; '))
df = df.explode('Drug IDs').dropna()
df.head()

In [ ]:
df.shape

 ## Map Drug IDs to Names

In [ ]:
df['Drug IDs'] = drug_meta.reindex(df['Drug IDs']).set_index(df.index)
df.head()

 # Filter Data

 ## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [ ]:
df = uf.mapgenesymbols(df, symbol_lookup)
df.shape

 # Analyze Data

 ## Create Binary Matrix

In [ ]:
binary_matrix = uf.createBinaryMatrix(df)
binary_matrix.head()

In [ ]:
binary_matrix.shape

In [ ]:
uf.saveData(binary_matrix, path, output_name + '_binary_matrix', 
            compression='npz', dtype=np.uint8)

 ## Create Gene List

In [ ]:
gene_list = uf.createGeneList(binary_matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.saveData(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

 ## Create Attribute List

In [ ]:
attribute_list = uf.createAttributeList(binary_matrix)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.saveData(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

 ## Create Gene and Attribute Set Libraries

In [ ]:
uf.createUpGeneSetLib(binary_matrix, path, output_name + '_gene_up_set')

In [ ]:
uf.createUpAttributeSetLib(binary_matrix, path, 
                           output_name + '_attribute_up_set')

 ## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.createSimilarityMatrix(binary_matrix.T, 'jaccard', sparse=True)
attribute_similarity_matrix.head()

In [ ]:
uf.saveData(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

In [ ]:
# net.load_df(attribute_similarity_matrix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
# net.cluster()
# net.widget()

 ## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.createSimilarityMatrix(binary_matrix, 'jaccard', sparse=True)
gene_similarity_matrix.head()

In [ ]:
uf.saveData(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

 ## Create Gene-Attribute Edge List

In [ ]:
uf.createGeneAttributeEdgeList(binary_matrix, attribute_list, gene_list, 
                               path, output_name + '_gene_attribute_edge_list')

 # Create Downloadable Save File

In [ ]:
uf.createArchive(path)

 ### Link to download output files: [click here](./output_archive.zip)